# [MinAbsSum](https://app.codility.com/programmers/lessons/17-dynamic_programming/min_abs_sum/)

- 答えは[ここ](https://codility.com/media/train/solution-min-abs-sum.pdf)

```
For a given array A of N integers and a sequence S of N integers from the set {−1, 1}, we define val(A, S) as follows:

val(A, S) = |sum{ A[i]*S[i] for i = 0..N−1 }|

(Assume that the sum of zero elements equals zero.)

For a given array A, we are looking for such a sequence S that minimizes val(A,S).

Write a function:

def solution(A)

that, given an array A of N integers, computes the minimum value of val(A,S) from all possible values of val(A,S) for all possible sequences S of N integers from the set {−1, 1}.

For example, given array:

  A[0] =  1
  A[1] =  5
  A[2] =  2
  A[3] = -2
your function should return 0, since for S = [−1, 1, −1, 1], val(A, S) = 0, which is the minimum possible value.

Write an efficient algorithm for the following assumptions:

N is an integer within the range [0..20,000];
each element of array A is an integer within the range [−100..100].
```

In [15]:
def minimum_val(arr, current_val):
    if len(arr) == 0:
        return 0
    if len(arr) == 1:
        return min(abs(current_val+arr[0]), abs(current_val-arr[0]))
    
    first_ele = arr[0]
    remaining_arr = arr[1:]
    
    min_possibility1 = minimum_val(remaining_arr, current_val+first_ele)
    if min_possibility1 == 0:
        return 0
    min_possibility2 = minimum_val(remaining_arr, current_val-first_ele)
    if min_possibility2 == 0:
        return 0
    return min(min_possibility1, min_possibility2)

def solution(A):
    return minimum_val(A, 0)

In [20]:
assert solution([-1, 1, -1, 1]) == 0
assert solution([5,-5,10,-10]) == 0
assert solution([5, -5, 10, -10, 15]) == 5

1. DPは将来的な結果を手繰り寄せながら`再帰的に`そのステップステップで最適な方を選んでいく
2. 計算時間が莫大になりうるので、`効率化(メモ化など)`しながら進める

と思っていたが、`思考を二次元に拡張してやったらメモ化をしなくても行ける`ことも

上記の解法だと`O(2^n)`の計算時間がかかるため、別の方法を使う。
その前にDPを勉強する

# [ The Coin Changing problem](https://codility.com/media/train/15-DynamicProgramming.pdf)

DPのお勉強。

- kで金額が与えられ、それを(`C[1,3,4]とか`)で与えられたコインの額のset内の効果を組み合わせて作る
- 最小枚数を求める

ちなみにDPはCがソートされている必要はない

In [54]:
import sys


def dynamic_coin_changing(c, k):
    # 効果の種類
    n = len(c)
    # tableの用意(row=使うコインの種類数[0-n枚], column=達成した金額[0-k円まで])
    dp = [[0]*(k+1) for _ in range(n+1)]
    
    dp[0] = [0] + [sys.maxsize] * k
    for num_coins in range(1, n+1):
        # それぞれの金額に関して、最大の硬貨の金額−1円までは、その上の列と同じ枚数
        # 最大金額の硬貨は使えないからね
        max_currency = c[num_coins-1]
        for amount in range(0, max_currency):
            dp[num_coins][:amount+1] = dp[num_coins-1][:amount+1]
        for amount in range(max_currency, k+1):
            dp[num_coins][amount] = min(dp[num_coins-1][amount], dp[num_coins][amount-num_coins]+1)
    # print(dp)
    return dp[n][k]
            
    

In [55]:
dynamic_coin_changing([1,3,4], 6)

[[0, 9223372036854775807, 9223372036854775807, 9223372036854775807, 9223372036854775807, 9223372036854775807, 9223372036854775807], [0, 1, 2, 3, 4, 5, 6], [0, 1, 2, 2, 3, 3, 4], [0, 1, 2, 2, 2, 3, 3]]


3

できた、、、発想はすごいが、実装は簡単。ちなみにこうやるとメモリ省略できる。

17.2: The dynamic algorithm for finding change with optimized memory.
```
def dynamic_coin_changing(C, k):
    n = len(C)
    dp = [0] + [MAX_INT] * k
    for i in xrange(1, n + 1):
        for j in xrange(C[i - 1], k + 1):
            dp[j] = min(dp[j - C[i - 1]] + 1, dp[j])
    return dp
```

ちゃんとDPを使って、MinAbsSumをもう一度とく
硬貨問題では最小枚数をDPテーブルに詰めて行ったが、この問題では、`そのcolumnのインデックスの値を作れるかの真偽値(0or1)`を収納していく

## Slow Solution

In [122]:
def solution(A):
    # まずは便宜上、全ての要素をその絶対値に置き換える
    for i in range(len(A)):
        A[i] = abs(A[i])
    S = sum(A)
    
    # 和が101なら、50までが和として実現可能かを調べる
    half_sum = S // 2
    num_ele = len(A)
    # どの要素も使わずに作れる和
    dp = [[0]*(half_sum+1) for _ in range(num_ele+1)]
    dp[0] = [1] + [0]*half_sum
    # for ith column...
    for i in range(1, num_ele+1):
        # (上の行で実現可能な値)か、(その和に今回新たに加わった数字を足した和)の場合は実現可能
        for j in range(half_sum+1):
            # 今回新たに加わった数字はA[i]ではなく、A[i-1]
#             print(j, A[i-1])
            if (dp[i-1][j] == 1) | ((j >= A[i-1]) and (dp[i-1][j-A[i-1]] == 1)):
                dp[i][j] = 1
#     for l in dp:
#         print(l)

    for i in range(half_sum, -1, -1):
        if dp[-1][i] == 1:
            return abs(S - 2 * i)
            
    

In [149]:
solution([1,3,9])

1

メモリオーバーしたので、dpを一行にする
- これでもtime complexityはO(N * (N * MaxOfArray / 2)) =  O(N**2 * M)のはず
- space complexityはO(N*M)のはず

In [132]:
def solution(A):
    # まずは便宜上、全ての要素をその絶対値に置き換える
    for i in range(len(A)):
        A[i] = abs(A[i])
    S = sum(A)
    
    # 和が101なら、50までが和として実現可能かを調べる
    half_sum = S // 2
    num_ele = len(A)
    # どの要素も使わずに作れる和
    dp = [1] + [0]*half_sum
    # for ith column...
    for i in range(1, num_ele+1):
        # (上の行で実現可能な値)か、(その和に今回新たに加わった数字を足した和)の場合は実現可能
        # 後者を利用するために、前の要素を変えないでおくために、後ろからループを回さないといけない
        for j in range(half_sum, -1, -1):
            # 今回新たに加わった数字はA[i]ではなく、A[i-1]
#             print(j, A[i-1])
            if (dp[j] == 1) | ((j >= A[i-1]) and (dp[j-A[i-1]] == 1)):
                dp[j] = 1
#     for l in dp:
#         print(l)

    for i in range(half_sum, -1, -1):
        if dp[i] == 1:
            return abs(S - 2 * i)

## 同じ要素が複数含まれていたら、まとめて同じ行内で処理する

In [172]:
def solution(A):
    # まずは便宜上、全ての要素をその絶対値に置き換える
    
    max_num = 0
    for i in range(len(A)):
        A[i] = abs(A[i])
        max_num = max(max_num, abs(A[i]))
    
    # 0-max_numについて、それぞれが含まれる個数を含むリスト
    count_of_num = [0] * (max_num+1)
    for num in A:
        count_of_num[num] += 1
    S = sum(A)
    
    # 和が101なら、50までが和として実現可能かを調べる
    half_sum = S // 2
    
    # どの要素も使わずに作れる和
    dp = [1] + [-1]*half_sum
    # 0-max_numまで走らせる。Aに入っていなかったものは飛ばされるので大丈夫。
    for num in range(1, max_num+1):
        if count_of_num[num] > 0:
            # (上の行で実現可能な値)か、(その和に今回新たに加わった数字を足した和)の場合は実現可能
            # これを実現するために、前者、つまりその上の行が実現可能だから実現可能という時は、
            # そこに、そこにその時に見ている数字iの存在回数を入れる。
            # 後者の場合は、前からtraverseして行って、i前のindexが1より大きければ実現可能なので、
            # i前のindex-1を入れる
            #
            # [○,-1,-1,○,○,-1,○]で、現在i=2を見ていてi=2が3回現れたのなら、、
            # ->[3,-1,-1,○,○,-1,○]
            # ->[3,-1,2,○,○,-1,○]
            # ->[3,-1,2,3,○,-1,○]
            # ->[3,-1,2,3,3,2,○]
            # ->[3,-1,2,3,3,2,2] となる
            #
            # このやり方だと、後ろからではなく、むしろ前から順々にテーブルの行を
            # 見ていかないといけない
            for j in range(half_sum+1):
                # 前者の場合
                if dp[j] >= 0:
                    dp[j] = count_of_num[num]
                elif (j >= num) and (dp[j-num] >= 1):
                    dp[j] = dp[j-num] - 1
#         print(num, dp)
    for i in range(half_sum, -1, -1):
        if dp[i] >= 0:
            return abs(S - 2 * i)

In [173]:
solution([1,3,9])

5

## できた。。。大変だった。。。

![image](https://user-images.githubusercontent.com/44487754/93112934-865dc700-f6f3-11ea-8dd5-51736219c741.png)


In [174]:
1

1

In [175]:
for i in None:
    print(i)

TypeError: 'NoneType' object is not iterable